In [ ]:
import pandas as pd
import datetime as dt
import numpy as np

In [ ]:
def tbillrate():
    df = pd.read_csv("data/TB3MS.csv")
    df["observation_date"] = pd.to_datetime(df["observation_date"])
    df["TB3MS"] /= 100
    return df.set_index("observation_date")
def goldprice():
    df = pd.read_csv("data/gldhist.csv").rename(columns={"price":"gold"})
    df["date"] = pd.to_datetime(df["date"])+dt.timedelta(days=1)
    return df.set_index("date")
def read_shiller():
    df = pd.read_csv("data/shiller.csv", header=[6])
    dflast = df.iloc[-1]
    df = df[:-1]
    df['Date'] = pd.to_datetime(["%.2f" % d for d in df['Date']], format="%Y.%m")#+dt.timedelta(days=14)
    columns={"Date":"Date","P":"SP500","D":"Div","E":"Earning","CPI":"CPI","Rate GS10":"Rate10y"}
    df.rename(columns=columns,inplace=True)
    df = df[columns.values()]
    df = df.set_index("Date")
    for c in df.columns:
        df[c] = df[c].astype(float)
    df['Rate10y'] = df['Rate10y']/100
    return df.join(tbillrate()).join(goldprice(),how="inner")

def compute_bond_tr(df):
    r = df['Rate10y']
    T = 10
    duration = -(1-np.exp(-r*T))/r
    bondcarry = r.shift(1)/12
    bondtotalret = 1+(r-r.shift(1))*duration+bondcarry
    df["bondTR"] = bondtotalret
def compute_eq_tr(df):
    df['eqTR'] = df['SP500']/df['SP500'].shift(1)+df['Div']/12/df['SP500']
def compute_gold_tr(df):
    df['gldTR'] = df['gold']/df['gold'].shift(1)
def compute_tbill_tr(df):
    df['tbTR'] = 1+df['TB3MS'].shift(1)/12
def compute_cpi_tr(df):
    df['cpiTR'] = 1+df['CPI'].pct_change()

    
df = read_shiller()
compute_bond_tr(df)
compute_eq_tr(df)
compute_gold_tr(df)
compute_tbill_tr(df)
compute_cpi_tr(df)
df.to_csv("totalreturns.csv")
df

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.plot(df['eqTR'].cumprod(), label="SP500 total return")
plt.plot(df["bondTR"].cumprod(), label="10y TNote total return")
plt.plot(df["gldTR"].cumprod(), label="Gold total return")
plt.plot(df["tbTR"].cumprod(), label="TBill total return")
plt.plot(df["cpiTR"].cumprod(), label="cpi total return")
plt.yscale('log')
plt.grid(which="both",axis="y")
plt.grid(axis="x")
plt.legend()
plt.xlabel("last dividend data on " + str(df["Div"].dropna().index[-1])[:10])
plt.ylabel("Log Nominal Wealth")
plt.title("Total Return (Log Scale)")
plt.show()

In [ ]:
trcols = [c for c in df.columns if "TR" in c]
logret = np.log(df[trcols])
mu    = np.mean(logret,axis=0)*12
sigma = np.std(logret,axis=0)*np.sqrt(12)
corr  = logret.corr()
paramsdf = pd.DataFrame({"mu":mu,"sigma":sigma},mu.index)
r       = paramsdf.loc["tbTR","mu"]
sigma_c = paramsdf.loc["cpiTR","sigma"]
paramsdf["mu-r"] = (paramsdf["mu"]-r)
paramsdf["sharpe_nominal"] = (paramsdf["mu-r"])/paramsdf["sigma"]
paramsdf["rho_c"] = [corr.loc[c,"cpiTR"] for c in trcols]
paramsdf["sharpe_real"] = (paramsdf["mu-r"])/paramsdf["sigma"] - paramsdf["rho_c"]*sigma_c
paramsdf

In [ ]:
corr

In [ ]:
# logret 1971-02-01 2025-10-01
logret.cov()*12

In [ ]:
# Covariance Matrix:
#            bonds    equity      gold
# bonds   0.004850  0.000449 -0.000472
# equity  0.000449  0.015726 -0.000576
# gold   -0.000472 -0.000576  0.029258


In [ ]:
df[trcols]